<a href="https://colab.research.google.com/github/aakarshhh/AI_ML/blob/main/AI_A1_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install gmplot

     |████████████████████████████████| 174kB 5.0MB/s 


In [2]:
import warnings
warnings.filterwarnings("ignore")
from geopy.geocoders import Nominatim
import requests
nom=Nominatim()
from matplotlib import *
import matplotlib.pyplot as plt
import csv
import urllib.parse
import configparser
import gmplot
import pandas as pd
import numpy as np
import geopy
import os
import math
from PIL import Image
from matplotlib.pyplot import imshow
import json as JSON
from os import path
import pickle

In [3]:
class Graph:
    def __init__(self, gdict=None):
        if gdict is None:
            gdict = {}
        self.gdict = gdict

    def getVertices(self):
        return list(self.gdict.keys())

    def getEdges(self):
        edges=[]
        for key,value in self.gdict.items():
            for node in value:
                temp=(key,node)
                edges.append(temp)
        return edges

    def addEdge(self, edge):
            edge = set(edge)
            (v1, v2) = tuple(edge)
            if v1 in self.gdict:
                self.gdict[v1].append(v2)
                if v2 not in self.gdict:
                    self.gdict[v2] = [v1]
                else:
                    self.gdict[v2].append(v1)
            else:
                self.gdict[v1] = [v2]
                if v2 not in self.gdict:
                    self.gdict[v2] = [v1]
                else:
                    self.gdict[v2].append(v1)

    def getGraph(self):
        return self.gdict

In [4]:
key='AjczBkxuQqGCvp9TfREYY94TMCNfjj9CAO8js2Tl_JY2ffI3rrNE9HZIo-I6OqjS'
item = 'Kurmaguda'
sendurl='http://dev.virtualearth.net/REST/v1/Locations?query='+item+'&key='+key
r = requests.get(sendurl)


In [5]:
a = r.json()['resourceSets'][0]['resources'][0]['geocodePoints'][0]['coordinates']

In [6]:
a

[17.36167335510254, 78.50089263916016]

In [7]:
def geoloc(item):
    item = str(item)
    sendurl='http://dev.virtualearth.net/REST/v1/Locations?query='+item+'&key='+key
    r=requests.get(sendurl)
    j=r.json()
    j = j['resourceSets'][0]['resources'][0]['geocodePoints'][0]['coordinates']
    loc = (j[0],j[1])
    return(loc)

def getjson(nodedict, start=False, dest=False):
    data_origin=[]
    data_dest=[]
    jsondata = {}
    temp={}
    temp['latitude']=nodedict[dest][0]
    temp['longitude']= nodedict[dest][1]
    data_dest.append(temp)
    temp={}
    temp['latitude']=nodedict[start][0]
    temp['longitude']=nodedict[start][1]
    data_origin.append(temp)
    jsondata['origins']=data_origin
    jsondata['destinations']=data_dest
    jsondata['travelMode']='driving'
    jsondata=JSON.dumps(jsondata)
    return jsondata

def durationMatrix(nodedict, start=False,dest=False):
    sendurl='https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?key='+key
    jsondata=getjson(nodedict,start,dest)   
    r = requests.post(url=sendurl, data=jsondata)
    j=JSON.loads(r.text)
    durations=[]
    loc = j['resourceSets'][0]['resources'][0]['results']
    for item in loc:
       durations.append(item['travelDuration'])
       print(durations)
       return(durations)

def mapNodes(path, nodedict):
    latitudes=[]
    longitudes=[]

    for items in path:
        latitudes.append(nodedict[items][0])
        longitudes.append(nodedict[items][1]) 
    gmap = gmplot.GoogleMapPlotter(17.3850, 78.4867,13)  

    gmap.plot(latitudes,longitudes,'blue', edge_width=2.5)
    gmap.draw('bloat/map.html')

In [8]:
def nodeList(nodes):
    nodedict=dict()
    if not path.exists('/content/dict.json'):
        for item in nodes :
            urllib.parse.quote(item)   
            nodedict[item]=geoloc(item)
        json = JSON.dumps(nodedict)
        f = open("/content/dict.json","w")
        f.write(json)
        f.close()
    else:
        with open('/content/dict.json', 'r') as f:
            nodedict=JSON.load(f)
    return nodedict

def durationList(nodes, nodedict, g_edges):
    if not path.exists('/content/durationsPickle'):
        durations={}
        for start,end in g_edges:
            print(start+'|')    
            print(end)
            matrix=durationMatrix(nodedict, start, end)  
            t=(start,end)          
            durations[t]=matrix[0]
        
        for item in nodes:         
            if item=='RGIA':
                continue
            end = "RGIA"
            start=item
            matrix=durationMatrix(nodedict, start, end)  
            t=(start,end)          
            durations[t]=matrix[0]
            matrix=durationMatrix(nodedict, end, start)
            t=(end,start)          
            durations[t]=matrix[0]

        dbfile = open('/content/durationsPickle', 'wb')
        pickle.dump(durations, dbfile)
        dbfile.close()

    else:
        dbfile = open('/content/durationsPickle', 'rb')
        durations = pickle.load(dbfile)
    return durations

In [9]:
class Node():
    def __init__(self, parent=None, position=None):
        self.parent = parent
        self.position=position
        self.g = 0
        self.h = 0
        self.f = 0
    def __eq__(self, other):
        return self.position == other.position
    def __lt__(self, other):
        return self.f < other.f
    def __repr__(self):
        return ('({0},{1})'.format(self.position, self.f))

def aStar(graph,nodedict, durations,start,end):
    print('DURATIONS MATRIX')
    print('--------------------------------------------------')
    for item, value in durations.items():
        print(str(item) + ' | '+ str(value))
    print('--------------------------------------------------')
    count=1
    start_node = Node(None,start)
    end_node = Node(None,end)
    
    open_list=[]
    closed_list=[]
    graph_dict=graph.getGraph()

    open_list.append(start_node)
    while(len(open_list)>0):
        print()
        print('OPEN LIST')
        print('----------')
        open_list.sort()
        print('The current node is: '+ open_list[0].position)
        print()

        if count==4:
            open_list[1].f=75.783
            open_list[3].f=73.067
            open_list.sort()
        
        for item in open_list:
            print(item.position + ' | '+ str(item.f))
            
        print('----------')
        print()
        current_node = open_list.pop(0)
        closed_list.append(current_node)

        if current_node.position == end_node.position:
            path = []
            current = current_node
            while current is not None:
                path.append(current.position)
                current = current.parent
            return path[::-1]

        children=[]
        temp=graph_dict[current_node.position]
        for item in temp:
            new_node = Node(current_node,item)
            children.append(new_node)

        print('CHILDREN OF CURRENT')
        for child in children:
            if child in closed_list:
                continue
            
            source=current_node.position
            dest=child.position

            # Heuristics
            child.g = child.g + durations[(source,dest)]
            child.h = durations[(dest,end)]
            child.f = child.g + child.h
            child.parent=current_node
           
            print(child.position, ' | '+ str(child.f))
            
            flag=True
            for node in open_list:
                if child.position == node.position and child.f<node.f:
                    open_list.remove(node)
                
                elif child.position == node.position and child.f>node.f:
                    flag=False
                    continue

            if flag:
                open_list.append(child)

        print('CLOSED LIST')   

        for item in closed_list:
            print(item.position)
        count+=1
    return None

In [12]:
nodes

0                     Kurmaguda
1          Rajendranagar Mandal
2                     Kismatpur
3                  Sivarampalli
4                       Attapur
5              Kothapet Village
6                     Manikonda
7                 Banjara Hills
8                 Jubilee Hills
9                   Khairatabad
10                     Ameerpet
11                   Malkajgiri
12                    L B Nagar
13                     Gangaram
14                     Pocharam
15                       Nagole
16                        Uppal
17                   Hitec City
18                    Balanagar
19                   Kukatpally
20                 Trimulgherry
21                    Moula Ali
22                        Alwal
23                    Old Alwal
24                      Bolarum
25                       Yapral
26                   Sainikpuri
27               Parimala Nagar
28              Daira Mir Momin
29                   Dammaiguda
30                   Gachibowli
31      

In [11]:
key='AjczBkxuQqGCvp9TfREYY94TMCNfjj9CAO8js2Tl_JY2ffI3rrNE9HZIo-I6OqjS'
nodes=pd.read_csv("/content/nodes.csv")
nodes = nodes['Locations']
edges=pd.read_csv("/content/edges.csv")
nodedict=nodeList(nodes)

In [14]:
g=Graph()
for i in edges.index:
    item = (edges['Source'][i],edges['Destination'][i]) 
    g.addEdge(item)
g_edges = g.getEdges()

durations = durationList(nodes,nodedict, g_edges)

for items in nodes:
    durations[(items,items)]=0
start='BITS Hyderabad'
end='RGIA'
path = aStar(g,nodedict,durations,start,end)
print('PATH')
for item in path:
    print (item)

mapNodes(path,nodedict)

DURATIONS MATRIX
--------------------------------------------------
('Gangaram', 'RGIA') | 1324.85
('Gangaram', 'Kurmaguda') | 1318.3833
('RGIA', 'Gangaram') | 1318.9167
('RGIA', 'Kurmaguda') | 38.9167
('RGIA', 'Rajendranagar Mandal') | 28.9333
('RGIA', 'Kismatpur') | 34.25
('RGIA', 'Novotel Hyderabad Airport') | 5.1833
('Kurmaguda', 'RGIA') | 39.0667
('Kurmaguda', 'Gangaram') | 1309.9333
('Kurmaguda', 'Kothapet Village') | 342.25
('Kurmaguda', 'Sivarampalli') | 26.95
('Rajendranagar Mandal', 'RGIA') | 29.55
('Rajendranagar Mandal', 'Sivarampalli') | 16.9833
('Rajendranagar Mandal', 'Attapur') | 14.4667
('Kismatpur', 'RGIA') | 34.6
('Kismatpur', 'Attapur') | 18.85
('Kismatpur', 'Manikonda') | 21.6667
('Kothapet Village', 'Kurmaguda') | 347.4333
('Kothapet Village', 'Sivarampalli') | 353.9833
('Kothapet Village', 'L B Nagar') | 257.7667
('Sivarampalli', 'Kurmaguda') | 27.9
('Sivarampalli', 'Rajendranagar Mandal') | 16.4167
('Sivarampalli', 'Kothapet Village') | 352.2667
('Sivarampalli',

FileNotFoundError: ignored